In [1]:
from sklearn.datasets import fetch_lfw_people

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [2]:
# fetch the dataset
lfw_people = fetch_lfw_people(min_faces_per_person=30g)

/home/dcs/csunbd/anaconda3/lib/python3.5/site-packages/sklearn/externals/joblib/func_inspect.py:191: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  arg_spec = inspect.getargspec(func)


In [3]:
from keras.layers.core import Activation, Dense, Dropout, Flatten

Using Theano backend.
ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
/home/dcs/csunbd/anaconda3/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [4]:
from keras.layers.convolutional import Convolution2D, MaxPooling2D

In [5]:
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical

In [6]:
y = lfw_people.target

In [7]:
X = lfw_people.data

In [8]:
sample, h, w = lfw_people.images.shape

In [9]:
from sklearn.preprocessing import normalize

In [10]:
X = normalize(X)

In [11]:
# Reshape into a 4D tensor
X = X.reshape(X.shape[0], 1, h, w)

In [13]:
# Now create the CNN
from keras.models import Sequential

In [26]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(1, h, w)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3, border_mode='valid'))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(32, 3, 3, border_mode='valid'))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3, border_mode='valid'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(len(lfw_people.target_names)))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer='adam')

In [14]:
from sklearn.cross_validation import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
y_train = to_categorical(y_train, len(lfw_people.target_names))

In [ ]:
model.fit(X_train, y_train, show_accuracy=True, nb_epoch=100, batch_size=128)

In [ ]:
y_predict = model.predict_classes(X_test, batch_size=128)

In [1]:
from sklearn.metrics import classification_report

In [2]:
print(classification_report(y_test, y_predict, target_names=lfw_people.target_names))